In [ ]:
from pathlib import Path

In [ ]:
from smtag.config import config
config.from_pretrained

In [ ]:
from transformers import __version__
__version__

## Extracting examples for TOKCL

In [ ]:
from smtag.extract import ExtractorXML

In [ ]:
corpus = "/data/xml/191012"
xml_examples = "/data/text/sd_test"
xpath = ".//sd-panel"
sentence_level = False
keep_xml = True
inclusion_probability = 0.1

In [ ]:
extractor_tokcl = ExtractorXML(
    corpus,
    destination_dir=xml_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

same via CLI:

```bash
python -m smtag.cli.prepro.extract /data/xml/191012/ /data/text/sd_test --xpath ".//sd-panel" --sentence_level --keep_xml --inclusion_probability 0.1
```

In [ ]:
extractor_tokcl.extract_from_corpus()

## Preparing tokenized dataset for TOKCL

In [ ]:
from smtag.dataprep import PreparatorTOKCL

In [ ]:
from smtag.xml2labels import SourceDataCodes as sd

In [ ]:
tokenized_examples = "/data/json/sd_test"
code_maps: code_maps = [
    sd.ENTITY_TYPES,
    sd.GENEPROD_ROLES,
    sd.BORING,
    sd.PANELIZATION
]
max_length = config.max_length

In [ ]:
prep_tokcl = PreparatorTOKCL(
    xml_examples,
    tokenized_examples,
    code_maps,
    max_length=max_length
)

In [ ]:
prep_tokcl.run()

same vie CLI:
    
```bash
python -m smtag.cli.tokcl.dataprep /data/text/sd_test /data/json/sd_test
```

## Train model for TOKCL

In [ ]:
from smtag.train.train_tokcl import (
    train as train_tokcl,
    TrainingArgumentsTOKCL
)

In [ ]:
training_args_tokcl = TrainingArgumentsTOKCL(
    num_train_epochs=1,
    logging_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
)
training_args_tokcl

In [ ]:
no_cache = False
loader_path = "./smtag/loader/loader_tokcl.py"
data_config_name =  "NER"
tokenizer = config.tokenizer
from_pretrained = "roberta-base"

In [ ]:
train_tokcl(
    training_args_tokcl,
    loader_path,
    data_config_name,
    tokenized_examples,
    no_cache,
    tokenizer,
    from_pretrained
)

same via CLI:
    
```bash
python -m smtag.cli.tokcl.train \
./smtag/loader/loader_tokcl.py NER \
--data_dir /data/json/sd_test \
--num_train_epochs=1 \
--logging_steps=100 \
--per_device_train_batch_size=16 \
--per_device_eval_batch_size=16
```